## In this notebook, we study the commits of x264 to get additional information about x264's features 

The commits are extracted from this repo : https://github.com/mirror/x264

We consider the master (or main?) branch 

Libraries

In [1]:
import os
import json

import numpy as np
import pandas as pd

#natural language processing
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
#nltk.download()

Load

In [3]:
commits = json.load(open("../../data/commits_x264/commits_x264.json"))

Example of a commit

In [4]:
commits[0]

{'sha': 'e586d699b2f13364aa443b367dba9fe38699f5de',
 'node_id': 'MDY6Q29tbWl0MjUwMTIxODk6ZTU4NmQ2OTliMmYxMzM2NGFhNDQzYjM2N2RiYTlmZTM4Njk5ZjVkZQ==',
 'commit': {'author': {'name': 'Fiona Glaser',
   'email': 'fiona@x264.com',
   'date': '2009-08-14T00:13:33Z'},
  'committer': {'name': 'Fiona Glaser',
   'email': 'fiona@x264.com',
   'date': '2009-08-14T00:13:33Z'},
  'message': 'Fix a potential failure in VBV\nIf VBV does underflow, ratecontrol could be permanently broken for the rest of the clip.\nRevert part of the previous VBV changes to fix this.',
  'tree': {'sha': 'b2fe82a234ece242d4710cf2d838e17ca3a8c869',
   'url': 'https://api.github.com/repos/mirror/x264/git/trees/b2fe82a234ece242d4710cf2d838e17ca3a8c869'},
  'url': 'https://api.github.com/repos/mirror/x264/git/commits/e586d699b2f13364aa443b367dba9fe38699f5de',
  'comment_count': 0,
  'verification': {'verified': False,
   'reason': 'unsigned',
   'signature': None,
   'payload': None}},
 'url': 'https://api.github.com/repos/m

## First of all, we delete all the commits posterior to the one relative to our version of x264

https://stackoverflow.com/questions/11856983/why-git-authordate-is-different-from-commitdate
We take the authordate as reference

In [12]:
msgs = pd.Series([c['commit']['committer']['email'] for c in commits]).unique()

len(msgs)
msgs

array(['fiona@x264.com', 'pengvado@akuvian.org', 'henrik@gramner.com',
       'BugMaster@narod.ru', 'pengvado@videolan.org',
       'fenrir@videolan.org', 'mru@mru.ath.cx', 'titer@videolan.org',
       'gpoirier@mplayerhq.hu', 'sam@videolan.org', 'robux@videolan.org',
       'chenm001@163.com', 'videolan@videolan.org', 'thresh@videolan.org',
       'lool@videolan.org', 'aizvorski@gmail.com', 'xtophe@videolan.org',
       'bugmaster@narod.ru', 'slyfox@inbox.ru', 'gramner@twoorioles.com',
       'fkuehne@videolan.org', 'eric.petit@lapsus.org'], dtype=object)

In [5]:
msgs = [c['commit']['message'] for c in commits if c['commit']['author']['date'][:10]<="2017-10-22"]

len(msgs)

2896

#### We keep 2896 commit messages

In [32]:
msgs[0:15]

['Fix a potential failure in VBV\nIf VBV does underflow, ratecontrol could be permanently broken for the rest of the clip.\nRevert part of the previous VBV changes to fix this.',
 'new API function x264_encoder_delayed_frames.\nfix x264cli on streams whose total length is less than the encoder latency.',
 'Add no-mbtree to fprofile (and fix pyramid in fprofile)',
 "Don't print a warning about direct=auto in 2pass when B-frames are off",
 'fix lowres padding, which failed to extrapolate the right side for some resolutions.\nfix a buffer overread in x264_mbtree_propagate_cost_sse2. no effect on actual behavior, only theoretical correctness.\nfix x264_slicetype_frame_cost_recalculate on I-frames, which previously used all 0 mb costs.\nshut up a valgrind warning in predict_8x8_filter_mmx.',
 'simd part of x264_macroblock_tree_propagate.\n1.6x faster on conroe.',
 'MB-tree fixes:\nAQ was applied inconsistently, with some AQed costs compared to other non-AQed costs. Strangely enough, fixing 

# Analyze:

## Performance

### Encoding size

## Features

### 1. mbtree
### 2. aq-mode
### 3. subme



### We:
- remove the punctuation;
- normalize the words; 
- remove the stopwords;
- use stemming & lemmatization.

In [42]:
stop_w = stopwords.words("english")
lemmatizer = WordNetLemmatizer()
ps = PorterStemmer()

keywords_size = ['size', 'byte', 'space', 'file', 'memori', 'output', 'video']

keywords_mbtree = ['mbtree','mb-tree','mb_tree', 'macroblock']
keywords_aq_mode = ['aq', 'aq-mode', 'aq_mode']
keywords_subme = ['subm', 'subq', 'subme']

#keywords_decrease = ['reduc', 'decreas', 'small', 'save' ]
#keywords_increase = ['larg', 'increas', ]

def get_commits(keywords, list_words):
    keep_commit = False
    for kw_mbt in keywords:
        for mw in list_words:
            if kw_mbt in list_words:
                keep_commit = True
    return keep_commit

commits_size = dict()
commits_mbt = dict()
commits_aq = dict()
commits_subme = dict()

for id_msg in range(len(msgs)):
    
    msg = msgs[id_msg].lower()
    
    #print(msg)
    
    del_c = ['*', '"', ',', ';', ':', '(', ')']
    space_c = ['\n', '.']

    for c in del_c:
        msg = msg.replace(c, '')
    for c in space_c:
        msg = msg.replace(c, ' ')

    words = msg.split(' ')

    # not used to stemmer, a bit strange
    main_words = [ps.stem(lemmatizer.lemmatize(w)) for w in words if w not in stop_w and w != '']

    # a shot without stemmer
    #main_words = [lemmatizer.lemmatize(w) for w in words if w not in stop_w and w != '']

    commits_size[id_msg] = get_commits(keywords_size, main_words)
    commits_mbt[id_msg] = get_commits(keywords_mbtree, main_words)
    commits_aq[id_msg] = get_commits(keywords_aq_mode, main_words)
    commits_subme[id_msg] = get_commits(keywords_subme, main_words)
        
    #print(main_words)


size_index = [c_index for c_index, c_bool in commits_size.items() if commits_size[c_index]]
print("Number of size commits :", len(size_index))

mbt_index = [c_index for c_index, c_bool in commits_mbt.items() if commits_mbt[c_index]]
print("Number of mbt commits :", len(mbt_index))

aq_index = [c_index for c_index, c_bool in commits_aq.items() if commits_aq[c_index]]
print("Number of aq commits :", len(aq_index))

subme_index = [c_index for c_index, c_bool in commits_subme.items() if commits_subme[c_index]]
print("Number of subme commits :", len(subme_index))

Number of size commits : 316
Number of mbt commits : 86
Number of aq commits : 14
Number of subme commits : 20


In [34]:
np.savetxt("../../data/commits_doc/size_commits.txt", 
           np.array([msgs[k] for k in size_index], dtype=str), 
           fmt='%s')

np.savetxt("../../data/commits_doc/mbtree_commits.txt", 
           np.array([msgs[k] for k in mbt_index], dtype=str), 
           fmt='%s')

np.savetxt("../../data/commits_doc/aq_commits.txt", 
           np.array([msgs[k] for k in aq_index], dtype=str), 
           fmt='%s')

np.savetxt("../../data/commits_doc/subme_commits.txt", 
           np.array([msgs[k] for k in subme_index], dtype=str), 
           fmt='%s')

In [35]:
commits[size_index[3]]

{'sha': '835ccc3cec908b1febfd31613d3e6583628116b3',
 'node_id': 'MDY6Q29tbWl0MjUwMTIxODk6ODM1Y2NjM2NlYzkwOGIxZmViZmQzMTYxM2QzZTY1ODM2MjgxMTZiMw==',
 'commit': {'author': {'name': 'Fiona Glaser',
   'email': 'fiona@x264.com',
   'date': '2009-08-05T00:46:33Z'},
  'committer': {'name': 'Fiona Glaser',
   'email': 'fiona@x264.com',
   'date': '2009-08-07T05:57:40Z'},
  'message': 'Macroblock-tree ratecontrol\nOn by default; can be turned off with --no-mbtree.\nUses a large lookahead to track temporal propagation of data and weight quality accordingly.\nRequires a very large separate statsfile (2 bytes per macroblock) in multi-pass mode.\nDoesn\'t work with b-pyramid yet.\nNote that MB-tree inherently measures quality different from the standard qcomp method, so bitrates produced by CRF may change somewhat.\nThis makes the "medium" preset a bit slower.  Accordingly, make "fast" slower as well, and introduce a new preset "faster" between "fast" and "veryfast".\nAll presets "fast" and above 

In [36]:
tab = np.array([msgs[k] for k in subme_index], dtype=str)

for i in range(len(tab)):
    print(i)
    print(commits[subme_index[i]]["commit"]["author"])
    print(tab[i])

0
{'name': 'Loren Merritt', 'email': 'pengvado@videolan.org', 'date': '2004-12-29T10:53:03Z'}
Chroma mode decision/subpel for B-frames
Improves compression ~0.4-1%. Helps more on videos with lots of chroma detail.
Enabled at subme 9 (preset slower) and higher.
1
{'name': 'Loren Merritt', 'email': 'pengvado@videolan.org', 'date': '2005-08-17T15:18:42Z'}
Full sub8x8 RD mode decision
Small speed penalty with p4x4 enabled, but significant quality gain at subme >= 6
As before, gain is proportional to the amount of p4x4 actually useful in a given input at the given bitrate.
2
{'name': 'Fiona Glaser', 'email': 'fiona@x264.com', 'date': '2008-11-04T06:59:49Z'}
Fix intra refresh with subme < 6
Also improve the quality of intra masking.
3
{'name': 'Fiona Glaser', 'email': 'fiona@x264.com', 'date': '2009-04-18T06:38:29Z'}
MBAFF: Disable adaptive MBAFF when subme 0 is used
4
{'name': 'Henrik Gramner', 'email': 'henrik@gramner.com', 'date': '2016-02-07T13:55:26Z'}
Further reduce code size in bime
~

In [41]:
pd.read_table("../../data/commits_doc/res.txt", delimiter="', '")

/home/llesoil/anaconda3/envs/x264/lib/python3.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,'message,author,date'
0,'Quality improvement from MB-tree is about 2-7...,Fiona Glaser,2009-08-05T00:46:33Z'
1,'Attempt to optimize PPS pic_init_qp in 2-pass...,Fiona Glaser,2012-09-26T20:49:02Z'
2,'Allow setting a chroma format at compile time...,Janne Grunau,2014-07-18T15:44:57Z'
3,'Chroma mode decision/subpel for B-frames. Imp...,Loren Merritt,2004-12-29T10:53:03Z'
4,'Make b-pyramid normal the default. Now that b...,Loren Merritt,2006-08-16T20:13:06Z'
5,'Add faster mv0 special case for macroblock-tr...,Guillaume Poirier,2006-11-27T21:40:21Z'
6,'CABAC trellis opts part 1: minor change in ou...,Nikoli,2011-04-29T10:19:22Z'
7,'Fix two bugs in 2-pass ratecontrol. last_qsca...,Alexandra Hájková,2016-11-14T14:06:06Z'
8,'finish subpixel motion refinement for B-frame...,Anton Mitrofanov,2013-05-22T18:43:59Z'
9,"'New ""superfast"" preset, much faster intra ana...",Loren Merritt,2005-10-08T06:49:29Z'
